# 実験パッケージの構成状態をモニタリングする

ここでは、作成いただいた実験パッケージの構成状態をモニタリングします。以下のセルを実行することで、それぞれの実験パッケージがその管理品質上、適合的であるかどうかを確認できます。モニタリングの結果は各実験パッケージのREADME.mdへ自動的に反映されます。

![result_on_README](../../../../files/WORKFLOWS/images/result_on_README.png)

研究ワークフロー機能トップページに戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この研究実行環境で操作する研究リポジトリ名を確認できます。  
このタスクでは、表示された名前の研究リポジトリ配下の実験パッケージについて構成状態のモニタリングを行います。  
研究リポジトリ名が表示されない場合は研究ワークフロー機能トップページのセクション「研究リポジトリ名を確認する」のセルを実行してください。  

In [ ]:
from IPython.display import display, HTML

# 研究リポジトリ名を表示する
%store -r
if 'RESEARCH_TITLE' not in locals().keys() : RESEARCH_TITLE = '-'
res_text = "<h1 style='color: green'>研究リポジトリ名：%s</h1>" % (RESEARCH_TITLE)
display(HTML(res_text))

## 1. 各実験パッケージのデータセット構成をモニタリングする

In [ ]:
import os
os.chdir(os.environ['HOME'])

# リポジトリと同期して内容を最新化する
!datalad update -s gin --merge

In [ ]:
import os
import json
from typing import List

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

def get_ideal_paths(scheme_name: str, path_to_dataset: str) -> List[str]:
    # 存在することが望ましいディレクトリ・ファイルのリスト
    ideal_paths_with_code = [
        path_to_dataset + '/source',
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/meta_data.json'
    ]

    # パラメータ実験毎に存在していほしいディレクトリを構成をチェックする方法を検討中。
    # まずは必ず存在するディレクトリをチェックする。
    ideal_paths_for_parameters = [
        path_to_dataset + '/source',
        path_to_dataset + '/input_data',
        path_to_dataset + '/meta_data.json'
    ]

    return eval('ideal_paths_' + scheme_name)

###########################################################################################################


def monitor_dataset_structure(scheme_name: str, paths_to_dataset: List[str]) -> bool:
    ideal_paths: List[str] = []
    dataset_name: str = ''

    for dataset in paths_to_dataset:
        isOK: bool = True
        ideal_paths = get_ideal_paths(scheme_name, dataset)
        dataset_name = os.path.basename(dataset)

        print('================================ Monitoring Dataset Structure ================================')
        print('Dataset name: ' + dataset_name)
        print(' STATUS    PATH')

        for path in ideal_paths:
            if os.path.exists(path):
                print('  OK    ' + path)
            else:
                print('  NG    ' + path)
                print('        [WARN] Your dataset structure is deprecated.')
                if isOK:
                    isOK = False

        print('\nMonitered by "' + scheme_name + '" scheme.\n')
        utils.reflect_monitoring_results('datasetStructure', isOK, dataset)

    return isOK

###########################################################################################################

# スクリプトファイルの拡張子リストを定義（あるいは読み出し）
# something cool...


def main() -> None:
    path_monitoring_params = utils.fetch_monitoring_param_file_path()

    # データセットのパスのリストを作成
    base_path = '/home/jovyan/experiments/'
    files = os.listdir(base_path)
    global paths_to_dataset
    paths_to_dataset = [base_path + f for f in files if os.path.isdir(os.path.join(base_path, f))]

    # データセット構成のスキームを取得
    scheme_name = ''
    with open(path_monitoring_params, "r") as f:
        dmp = json.load(f)
        scheme_name = dmp["datasetStructure"]

    isOK: bool = monitor_dataset_structure(scheme_name, paths_to_dataset)

if __name__ == '__main__':
    main()


## 2. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

git_path=[]
for path in paths_to_dataset:
    git_path.append(path+'/README.md')
git_path.append('WORKFLOWS/FLOW/02_experimental_phase/base_monitor_dataset_structure.ipynb')

utils.syncs_with_repo(git_path, gitannex_path=None, message='モニタリング（データセット構成）')

## 3. 研究ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究ワークフロー機能トップページに戻ってください。  

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))